In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
raw_df_match_details = pd.read_csv('../data/raw/match_details.csv')
raw_df_player_attr = pd.read_csv('../data/raw/player_attributes.csv')

In [3]:
raw_df_player_attr

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,183974,102359,39902,2009-08-30 00:00:00,83.0,85.0,right,medium,low,84.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,183975,102359,39902,2009-02-22 00:00:00,78.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,183976,102359,39902,2008-08-30 00:00:00,77.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30 00:00:00,78.0,81.0,right,medium,low,74.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [4]:
# raw_df_match_details.info()

In [5]:
# raw_df_player_attr.info()

In [6]:
raw_df_match_details = (raw_df_match_details
 .assign(date=pd.to_datetime(raw_df_match_details['date']),
         result_match=lambda df: (df['result_match'] == 'H').astype(int))
 )

In [7]:
raw_df_match_details['date'] = pd.to_datetime(raw_df_match_details['date'])
raw_df_player_attr['date'] = pd.to_datetime(raw_df_player_attr['date'])
raw_df_match_details.sort_values(by=['season', 'date'],inplace=True)
# raw_df_match_details.drop('stage', axis=1, inplace=True)

In [8]:
from playerstats import player_stats

players_cols = ['{}_player_{}'.format(team, i) for team in ['home', 'away'] for i in range(1, 12)]

player_stats_dict_series = raw_df_match_details.apply(
    lambda row: player_stats.get_player_stat(
        match_row=row,
        df_matches=raw_df_match_details,
        df_player_attr=raw_df_player_attr,
        players=players_cols
    ),
    axis=1
)

new_player_stats_df = pd.json_normalize(player_stats_dict_series)

df = pd.merge(raw_df_match_details, new_player_stats_df, how='left', on='match_api_id')
df.drop(players_cols, axis=1, inplace=True)
matching_columns = [col for col in df.columns if 'player_rating' in col] + [col for col in df.columns if 'aggression_rating' in col] + [col for col in df.columns if 'strength_rating' in col] + [col for col in df.columns if 'acceleration_rating' in col]
df.dropna(subset=matching_columns, inplace=True)
matching_columns_filtered = [c for c in df.columns if c != "date" and c != "season"]

In [9]:
df_=df.copy()

In [10]:
from src.preprocessing.shift_data import ShiftDataPreprocessor

preprocessor = ShiftDataPreprocessor(df_)

home_df = preprocessor.select_and_rename_columns('home_')
away_df = preprocessor.select_and_rename_columns('away_')

team_df = preprocessor.concatenate_teams(home_df, away_df)

features_to_shift = ['team_shoton', 'team_possession']
team_df_shifted = preprocessor.shift_features(features_to_shift)

In [11]:
# print(team_df_shifted.head())

In [12]:
df_final = preprocessor.merge_shifted_features(team_df_shifted)

INFO: Merging shifted features back into the original DataFrame.


In [14]:
df_final[['match_api_id', 'stage', 'date', 'home_team', 'away_team', 'home_prev_team_possession_shifted', 'away_prev_team_possession_shifted']].loc[(df_final.home_team == 8462) | (df_final.away_team == 8462)]

,match_api_id,stage,date,home_team,away_team,home_prev_team_possession_shifted,away_prev_team_possession_shifted
8,489051,1,2008-08-17,8455,8462,NaN,NaN
17,489059,2,2008-08-25,8462,10260,NaN,NaN
20,489067,3,2008-08-30,8668,8462,49.0,65.0
33,489079,4,2008-09-13,8462,8549,64.0,57.0
43,489086,5,2008-09-21,8456,8462,46.0,59.0
...,...,...,...,...,...,...,...
712,659151,34,2010-04-14,8528,8462,50.0,55.0
723,659163,35,2010-04-18,8462,10252,53.0,45.0
730,659176,36,2010-04-24,8559,8462,42.0,56.0
738,659183,37,2010-05-01,8462,8602,55.0,54.0


In [15]:
df_[['match_api_id', 'stage', 'date', 'home_team', 'away_team', 'home_possession', 'away_possession']].loc[(df_.home_team == 8462) | (df_.away_team == 8462)]

,match_api_id,stage,date,home_team,away_team,home_possession,away_possession
9,489051,1,2008-08-17,8455,8462,0,0
19,489059,2,2008-08-25,8462,10260,65,45
22,489067,3,2008-08-30,8668,8462,47,64
35,489079,4,2008-09-13,8462,8549,59,53
45,489086,5,2008-09-21,8456,8462,48,0
...,...,...,...,...,...,...,...
714,659151,34,2010-04-14,8528,8462,56,53
725,659163,35,2010-04-18,8462,10252,56,51
732,659176,36,2010-04-24,8559,8462,56,55
740,659183,37,2010-05-01,8462,8602,54,48
